# imports

In [1]:
from bs4 import BeautifulSoup
from datetime import date
from requests import get
import pandas as pd
today = date.today()

# crawler

## defining url

In [2]:
url = 'https://www.pararius.com/apartments/nederland/page-'
response = get(url+'1')
html_soup = BeautifulSoup(response.text, 'html.parser')

## get numbers to define iteractions

In [3]:
num_pages = html_soup.find_all('a',{"class":"pagination__link"})[4].text
num_pages = int(num_pages)

num_rents = html_soup.find_all('div',{"class":"pagination__summary"})[0].text
num_rents = int(num_rents.split()[-2])

In [4]:
print(url+'1')
print(num_pages)
print(num_rents)

https://www.pararius.com/apartments/nederland/page-1
177
5307


## get total html soup

In [5]:
houses = []
for count in range(1, num_pages+1):
    try:
        response = get(url + str(count))
        html_soup = BeautifulSoup(response.text, 'html.parser')
        house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")
        houses.append(house_data)
    except:
        pass

## get data

In [6]:
# description_=[]
price_=[]
address_=[]
street_=[]
agency_=[]
irl_=[]
image_=[]
description_2=[]

for n_pages in range (num_pages):
    n_pages_len = len(houses[n_pages])
    for n_rents in range (n_pages_len):
        num = (houses[n_pages][n_rents])
        list_li = num.find_all('li',{"class":"illustrated-features__item"})
        description_1={}
        
        try:
            p5_value = num.find_all('span',{"class":"listing-label listing-label--under-option"})[0].text
        except:
            p5_value = None
        
        for i in range(len(list_li)):
            a = (list_li[i]["class"][1][28:])
            b = (list_li[i].text)
            description_1.update({
                a:b,
                'status':p5_value,
            })
        description_2.append(description_1)
        
        price = num.find_all('div',{"class":"listing-search-item__price"})[0].text
        price_.append(price)

        address = num.find('div',{"class":"listing-search-item__location"}).text
        address_.append(address)

        street = num.find_all('a',{"class":"listing-search-item__link listing-search-item__link--title"})[0].text
        street_.append(street)

        irl = num.find_all('a',{"class":"listing-search-item__link listing-search-item__link--title"})[0]['href']
        irl_.append(irl)
        
        image = num.find_all('img')[0]['src']
        image_.append(image)

        agency = num.find_all('a', href=True)[2].text
        agency_.append(agency)

# Into DataFrame

In [7]:
df_pararius_1 = pd.DataFrame({
    'price':price_,
    'address':address_,
    'street':street_,
    'agency':agency_,
    'irl':irl_,
    'image':image_,
})
df_pararius_2 = pd.DataFrame(description_2)

## Unify dataframes

In [8]:
df_pararius = pd.concat([df_pararius_1, df_pararius_2],1)

<ipython-input-8-22791b88bcc6>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_pararius = pd.concat([df_pararius_1, df_pararius_2],1)


# fixing text

In [9]:
df_pararius['surface-area']        = df_pararius['surface-area'].str.replace("\D","",regex=True)
df_pararius['number-of-rooms']     = df_pararius['number-of-rooms'].str.replace("\D","",regex=True)
df_pararius['garden-surface-area'] = df_pararius['garden-surface-area'].str.replace("\D","",regex=True)
df_pararius['plot-size']           = df_pararius['plot-size'].str.replace("\D","",regex=True)

In [10]:
df_pararius = df_pararius.dropna(axis=1, how='all')
df_pararius['price'] = df_pararius['price'].str.replace("\D","",regex=True)
df_pararius['address'] = df_pararius['address'].str.replace('\n|new|  ',"",regex=True)
df_pararius['postcode'] = df_pararius['address'].str.replace("\s","",regex=True).str[0:6]
df_pararius['status'] = df_pararius['status'].str.replace('\n',"",regex=True)
df_pararius['date'] = str(today)

In [11]:
df_pararius = df_pararius.drop_duplicates(subset=['irl']).reset_index(drop=True)

# save data

In [12]:
today_csv = ("df_pararius_"+ str(today)+'.csv')
df_pararius.to_csv(f'data/temp/{today_csv}')

In [13]:
df_pararius

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,construction-period,interior,garden-surface-area,plot-size,postcode,date
0,1275,2522 VD Den Haag (Laakkwartier-Oost),Apartment Koopmans van Boekerenstraat,Level 5 Rental Agency,/apartment-for-rent/den-haag/6de86096/koopmans...,https://casco-media-prod.global.ssl.fastly.net...,59,None,4,1931,NaN,NaN,NaN,2522VD,2021-10-21
1,1650,1063 GB Amsterdam (Slotermeer-Noordoost),Apartment Arondeusstraat 1,Van Huis Uit Makelaars,/apartment-for-rent/amsterdam/97b8b421/arondeu...,https://casco-media-prod.global.ssl.fastly.net...,120,None,2,NaN,Upholstered,NaN,NaN,1063GB,2021-10-21
2,900,2311 HM Leiden (Levendaal-Oost),Studio Hogewoerd,RE/MAX Makelaarsgilde Leiden,/studio-for-rent/leiden/fceda9bf/hogewoerd,https://media.pararius.nl/image/PR0001659000/P...,56,None,1,NaN,Upholstered,NaN,NaN,2311HM,2021-10-21
3,725,3033 BD Rotterdam (Agniesebuurt),Studio Schiekade 47 K013,Wonen Hartje -,/studio-for-rent/rotterdam/c01aa36d/schiekade,https://media.pararius.nl/image/PR0001659000/P...,20,None,1,NaN,Upholstered or furnished,NaN,NaN,3033BD,2021-10-21
4,795,2313 KJ Leiden (Rijndijkbuurt),Apartment Hoge Rijndijk,RE/MAX Makelaarsgilde Leiden,/apartment-for-rent/leiden/1d38dd72/hoge-rijndijk,https://media.pararius.nl/image/PR0001659000/P...,26,None,2,NaN,Upholstered,NaN,NaN,2313KJ,2021-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5301,1500,1505 TJ Zaandam (Bomenbuurt),House Lijsterbesstraat,Uw Partner in Onroerende Zaken,/house-for-rent/zaandam/1b23bcc5/lijsterbesstraat,https://casco-media-prod.global.ssl.fastly.net...,86,Under option,4,NaN,Upholstered,NaN,NaN,1505TJ,2021-10-21
5302,3450,3045 LJ Rotterdam (Zestienhoven),House Overschiese Kleiweg,BenHousing,/house-for-rent/rotterdam/e5d0ae4e/overschiese...,https://casco-media-prod.global.ssl.fastly.net...,294,Under option,6,NaN,Upholstered or furnished,NaN,NaN,3045LJ,2021-10-21
5303,960,3762 GK Soest (Klaarwater),Apartment Wiardi Beckmanstraat,Regio Vastgoedmakelaars,/apartment-for-rent/soest/2e33b708/wiardi-beck...,https://casco-media-prod.global.ssl.fastly.net...,90,Under option,4,NaN,Shell,NaN,NaN,3762GK,2021-10-21
5304,1350,1567 LD Assendelft (Parkrijk),Apartment Wisselwachter,Uw Partner in Onroerende Zaken,/apartment-for-rent/assendelft/829433fa/wissel...,https://casco-media-prod.global.ssl.fastly.net...,76,Under option,3,NaN,Upholstered,NaN,NaN,1567LD,2021-10-21
